# Difference-in-Differences Analyse: Bouwsector Oprichtingen

Deze notebook voert een difference-in-differences (DiD) analyse uit om het effect van een treatment in 2019 op het aantal bedrijfsoprichtingen in de bouwsector in Vlaanderen te onderzoeken.

**Treatment definitie:**
- Treatment jaar: 2019
- Treatment regio: Vlaams Gewest
- Treatment sector: F Bouwnijverheid
- Outcome variabele: Aantal oprichtingen

## 1. Configuratie en Data Setup

In [72]:
# Import benodigde libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import warnings
warnings.filterwarnings('ignore')

# Zet pandas opties voor betere weergave
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [73]:
# CONFIGURATIE PARAMETERS VOOR DiD ANALYSE

# Treatment definitie
TREATMENT_YEAR = 2019
TREATMENT_REGION = "Vlaams Gewest"
TREATMENT_SECTOR = "F Bouwnijverheid"
OUTCOME_VARIABLE = "Aantal oprichtingen"

# Controle groepen
CONTROL_REGIONS = ["Brussels Hoofdstedelijk Gewest", "Waals Gewest"]

# Control sectoren: ALLE sectoren behalve bouwsector
# Eerst alle sectoren ophalen uit de data
all_unique_sectors = sorted([sector for sector in df['Sectie'].unique() if sector is not None])
CONTROL_SECTORS = [sector for sector in all_unique_sectors if sector != TREATMENT_SECTOR]

# Analyse periode
PRE_TREATMENT_YEARS = [2015, 2016, 2017, 2018]
POST_TREATMENT_YEARS = [2019, 2020, 2021, 2022]
ALL_YEARS = PRE_TREATMENT_YEARS + POST_TREATMENT_YEARS

# Data bestand
DATA_FILE = "data/jaar_alle_sectoren.json"

print(f"DiD Analyse Configuratie:")
print(f"- Treatment jaar: {TREATMENT_YEAR}")
print(f"- Treatment regio: {TREATMENT_REGION}")
print(f"- Treatment sector: {TREATMENT_SECTOR}")
print(f"- Outcome variabele: {OUTCOME_VARIABLE}")
print(f"- Controle regio's: {CONTROL_REGIONS}")
print(f"- Controle sectoren: ALLE SECTOREN behalve bouw ({len(CONTROL_SECTORS)} sectoren)")
print(f"- Analyse periode: {min(ALL_YEARS)}-{max(ALL_YEARS)}")

print(f"\nControl sectoren lijst:")
for i, sector in enumerate(CONTROL_SECTORS, 1):
    print(f"  {i:2d}. {sector}")

DiD Analyse Configuratie:
- Treatment jaar: 2019
- Treatment regio: Vlaams Gewest
- Treatment sector: F Bouwnijverheid
- Outcome variabele: Aantal oprichtingen
- Controle regio's: ['Brussels Hoofdstedelijk Gewest', 'Waals Gewest']
- Controle sectoren: ALLE SECTOREN behalve bouw (21 sectoren)
- Analyse periode: 2015-2022

Control sectoren lijst:
   1. A Landbouw, bosbouw en visserij
   2. B Winning van delfstoffen
   3. C Industrie
   4. D Productie en distributie van elektriciteit, gas, stoom en gekoelde lucht
   5. E Distributie van water/ afval- en afvalwaterbeheer en sanering
   6. G Groot- en detailhandel/ reparatie van auto’s en motorfietsen
   7. H Vervoer en opslag
   8. I Verschaffen van accommodatie en maaltijden
   9. J Informatie en communicatie
  10. K Financiële activiteiten en verzekeringen
  11. L Exploitatie van en handel in onroerend goed
  12. M Vrije beroepen en wetenschappelijke en technische activiteiten
  13. N Administratieve en ondersteunende diensten
  14. O Op

In [74]:
# Bekijk eerst alle beschikbare sectoren in de data
print("Alle beschikbare sectoren in de data:")
unique_sectors = sorted([sector for sector in df['Sectie'].unique() if sector is not None])
for i, sector in enumerate(unique_sectors, 1):
    print(f"{i:2d}. {sector}")
    
print(f"\nTotaal aantal sectoren: {len(unique_sectors)}")
print(f"Treatment sector: {TREATMENT_SECTOR}")

# Definieer control sectoren als alle sectoren behalve bouw
control_sectors_all = [sector for sector in unique_sectors if sector != TREATMENT_SECTOR]
print(f"\nControl sectoren (alle behalve bouw): {len(control_sectors_all)} sectoren")

Alle beschikbare sectoren in de data:
 1. A Landbouw, bosbouw en visserij
 2. B Winning van delfstoffen
 3. C Industrie
 4. D Productie en distributie van elektriciteit, gas, stoom en gekoelde lucht
 5. E Distributie van water/ afval- en afvalwaterbeheer en sanering
 6. F Bouwnijverheid
 7. G Groot- en detailhandel/ reparatie van auto’s en motorfietsen
 8. H Vervoer en opslag
 9. I Verschaffen van accommodatie en maaltijden
10. J Informatie en communicatie
11. K Financiële activiteiten en verzekeringen
12. L Exploitatie van en handel in onroerend goed
13. M Vrije beroepen en wetenschappelijke en technische activiteiten
14. N Administratieve en ondersteunende diensten
15. O Openbaar bestuur en defensie/ verplichte sociale verzekeringen
16. Onbekende economische activiteit
17. P Onderwijs
18. Q Menselijke gezondheidszorg en maatschappelijke dienstverlening
19. R Kunst, amusement en recreatie
20. S Overige diensten
21. T Huishoudens als werkgever/ niet-gedifferentieerde productie van goed

## 2. Data Loading en Preprocessing

In [75]:
# Laad de JSON data
with open(DATA_FILE, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Converteer naar DataFrame
df = pd.DataFrame(data['facts'])

print(f"Totaal aantal records geladen: {len(df)}")
print(f"Kolommen: {list(df.columns)}")
print(f"\nEerste 5 records:")
df.head()

Totaal aantal records geladen: 1955
Kolommen: ['Alle beschikbare periodes', 'Jaar', 'Gewest', 'Alle economische activiteiten', 'Sectie', 'Aantal btw-plichtige', 'Aantal oprichtingen', 'Aantal schrappingen']

Eerste 5 records:


,Alle beschikbare periodes,Jaar,Gewest,Alle economische activiteiten,Sectie,Aantal btw-plichtige,Aantal oprichtingen,Aantal schrappingen
0,Alle beschikbare periodes,2008,Vlaams Gewest,Alle economische activiteiten,"A Landbouw, bosbouw en visserij",34385.0,1326.0,2193.0
1,Alle beschikbare periodes,2008,Vlaams Gewest,Alle economische activiteiten,B Winning van delfstoffen,92.0,1.0,7.0
2,Alle beschikbare periodes,2008,Vlaams Gewest,Alle economische activiteiten,C Industrie,29740.0,1786.0,1502.0
3,Alle beschikbare periodes,2008,Vlaams Gewest,Alle economische activiteiten,"D Productie en distributie van elektriciteit, ...",135.0,19.0,6.0
4,Alle beschikbare periodes,2008,Vlaams Gewest,Alle economische activiteiten,E Distributie van water/ afval- en afvalwaterb...,837.0,71.0,30.0


In [76]:
# Data preprocessing
# Converteer jaar naar numeriek
df['Jaar'] = pd.to_numeric(df['Jaar'])

# Filter op relevante jaren
df_filtered = df[df['Jaar'].isin(ALL_YEARS)].copy()

# Filter op relevante regio's (treatment + control)
all_regions = [TREATMENT_REGION] + CONTROL_REGIONS
df_filtered = df_filtered[df_filtered['Gewest'].isin(all_regions)]

# Filter op relevante sectoren (treatment + control)
all_sectors = [TREATMENT_SECTOR] + CONTROL_SECTORS
df_filtered = df_filtered[df_filtered['Sectie'].isin(all_sectors)]

# Verwijder records met missing values in outcome variabele
df_filtered = df_filtered.dropna(subset=[OUTCOME_VARIABLE])

print(f"Data na filtering: {len(df_filtered)} records")
print(f"Periode: {df_filtered['Jaar'].min()}-{df_filtered['Jaar'].max()}")
print(f"Regio's: {sorted(df_filtered['Gewest'].unique())}")
print(f"Sectoren: {sorted(df_filtered['Sectie'].unique())}")

Data na filtering: 525 records
Periode: 2015.0-2022.0
Regio's: ['Brussels Hoofdstedelijk Gewest', 'Vlaams Gewest', 'Waals Gewest']
Sectoren: ['A Landbouw, bosbouw en visserij', 'B Winning van delfstoffen', 'C Industrie', 'D Productie en distributie van elektriciteit, gas, stoom en gekoelde lucht', 'E Distributie van water/ afval- en afvalwaterbeheer en sanering', 'F Bouwnijverheid', 'G Groot- en detailhandel/ reparatie van auto’s en motorfietsen', 'H Vervoer en opslag', 'I Verschaffen van accommodatie en maaltijden', 'J Informatie en communicatie', 'K Financiële activiteiten en verzekeringen', 'L Exploitatie van en handel in onroerend goed', 'M Vrije beroepen en wetenschappelijke en technische activiteiten', 'N Administratieve en ondersteunende diensten', 'O Openbaar bestuur en defensie/ verplichte sociale verzekeringen', 'Onbekende economische activiteit', 'P Onderwijs', 'Q Menselijke gezondheidszorg en maatschappelijke dienstverlening', 'R Kunst, amusement en recreatie', 'S Overige d

## 3. Exploratory Data Analysis

In [77]:
# Algemene statistieken
print("Beschrijvende statistieken voor Aantal oprichtingen:")
print(df_filtered[OUTCOME_VARIABLE].describe())

# Verdeling per regio
print("\nGemiddeld aantal oprichtingen per regio:")
regio_stats = df_filtered.groupby('Gewest')[OUTCOME_VARIABLE].agg(['mean', 'std', 'count'])
print(regio_stats)

# Verdeling per sector
print("\nGemiddeld aantal oprichtingen per sector:")
sector_stats = df_filtered.groupby('Sectie')[OUTCOME_VARIABLE].agg(['mean', 'std', 'count'])
print(sector_stats)

Beschrijvende statistieken voor Aantal oprichtingen:
count      525.000000
mean      1504.710476
std       2364.955618
min          0.000000
25%         27.000000
50%        667.000000
75%       1895.000000
max      14270.000000
Name: Aantal oprichtingen, dtype: float64

Gemiddeld aantal oprichtingen per regio:
                                       mean          std  count
Gewest                                                         
Brussels Hoofdstedelijk Gewest   541.028409   743.861081    176
Vlaams Gewest                   2837.102273  3426.303278    176
Waals Gewest                    1129.606936  1274.180889    173

Gemiddeld aantal oprichtingen per sector:
                                                           mean          std  \
Sectie                                                                         
A Landbouw, bosbouw en visserij                      616.625000   442.717872   
B Winning van delfstoffen                              3.083333     2.412227   
C In

In [ ]:
# Trend analyse over tijd
yearly_trends = df_filtered.groupby(['Jaar', 'Gewest', 'Sectie'])[OUTCOME_VARIABLE].sum().reset_index()

# Interactieve plot van trends per regio-sector combinatie
yearly_trends['Regio_Sector'] = yearly_trends['Gewest'] + ' - ' + yearly_trends['Sectie'].str.split(' ').str[0]

fig = px.line(yearly_trends, 
              x='Jaar', 
              y=OUTCOME_VARIABLE, 
              color='Regio_Sector',
              title='Trends in Aantal Oprichtingen per Regio-Sector (2015-2022)',
              markers=True)

# Voeg verticale lijn toe voor treatment jaar
fig.add_vline(x=TREATMENT_YEAR, line_dash="dash", line_color="red", 
              annotation_text=f"Treatment: {TREATMENT_YEAR}")

fig.update_layout(height=600, width=1000)
fig.show()

## 4. Treatment en Control Group Definitie

In [79]:
# Creëer indicator variabelen voor DiD analyse
df_did = df_filtered.copy()

# Post-treatment indicator (na 2019)
df_did['post_treatment'] = (df_did['Jaar'] >= TREATMENT_YEAR).astype(int)

# Treatment regio indicator (Vlaanderen)
df_did['treatment_region'] = (df_did['Gewest'] == TREATMENT_REGION).astype(int)

# Treatment sector indicator (Bouw)
df_did['treatment_sector'] = (df_did['Sectie'] == TREATMENT_SECTOR).astype(int)

# Treatment groep indicator (Vlaanderen EN Bouw)
df_did['treatment_group'] = (df_did['treatment_region'] & df_did['treatment_sector']).astype(int)

# DiD groepen definiëren
def get_group_label(row):
    if row['treatment_region'] == 1 and row['treatment_sector'] == 1:
        return 'Vlaanderen-Bouw'
    elif row['treatment_region'] == 1 and row['treatment_sector'] == 0:
        return 'Vlaanderen-Alle sectoren'
    elif row['treatment_region'] == 0 and row['treatment_sector'] == 1:
        return 'Bru/Wal-Bouw'
    else:
        return 'Bru/Wal-Alle sectoren'

df_did['group_label'] = df_did.apply(get_group_label, axis=1)

# Overzicht van groepen
print("DiD Groepen overzicht:")
group_summary = df_did.groupby(['group_label', 'post_treatment']).agg({
    OUTCOME_VARIABLE: ['mean', 'count'],
    'Jaar': ['min', 'max']
}).round(2)
print(group_summary)

DiD Groepen overzicht:
                                        Aantal oprichtingen          Jaar  \
                                                       mean count     min   
group_label              post_treatment                                     
Bru/Wal-Alle sectoren    0                           730.98   165  2015.0   
                         1                           790.96   168  2019.0   
Bru/Wal-Bouw             0                          2387.62     8  2015.0   
                         1                          2256.25     8  2019.0   
Vlaanderen-Alle sectoren 0                          2231.83    84  2015.0   
                         1                          2878.48    84  2019.0   
Vlaanderen-Bouw          0                          6143.00     4  2015.0   
                         1                         11373.00     4  2019.0   

                                                 
                                            max  
group_label              post

## 5. Difference-in-Differences Analyse

In [80]:
# Bereken gemiddeldes voor DiD berekening
did_means = df_did.groupby(['treatment_group', 'post_treatment'])[OUTCOME_VARIABLE].mean().unstack()

print("DiD Gemiddeldes Matrix:")
print("Rijen: Treatment Group (0=Control, 1=Treatment)")
print("Kolommen: Post Treatment (0=Pre, 1=Post)")
print(did_means.round(2))

# Bereken de verschillen
# Treatment groep: verschil voor vs na
treatment_diff = did_means.loc[1, 1] - did_means.loc[1, 0]
print(f"\nTreatment groep verschil (na - voor): {treatment_diff:.2f}")

# Control groep: verschil voor vs na
control_diff = did_means.loc[0, 1] - did_means.loc[0, 0]
print(f"Control groep verschil (na - voor): {control_diff:.2f}")

# DiD effect: verschil van verschillen
did_effect = treatment_diff - control_diff
print(f"\nDiD Effect (Treatment - Control verschil): {did_effect:.2f}")

# Percentage effect
baseline_treatment = did_means.loc[1, 0]
percentage_effect = (did_effect / baseline_treatment) * 100
print(f"DiD Effect als percentage van baseline: {percentage_effect:.2f}%")

DiD Gemiddeldes Matrix:
Rijen: Treatment Group (0=Control, 1=Treatment)
Kolommen: Post Treatment (0=Pre, 1=Post)
post_treatment        0         1
treatment_group                  
0                1273.1   1510.47
1                6143.0  11373.00

Treatment groep verschil (na - voor): 5230.00
Control groep verschil (na - voor): 237.38

DiD Effect (Treatment - Control verschil): 4992.62
DiD Effect als percentage van baseline: 81.27%


In [81]:
# Regressie analyse voor statistische significantie
# DiD regressie: Y = α + β1*Post + β2*Treatment + β3*(Post*Treatment) + ε

# Creëer interactie term
df_did['post_x_treatment'] = df_did['post_treatment'] * df_did['treatment_group']

# Fix voor kolom naam met spaties - gebruik Q() functie
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf

# Alternatief: gebruik directe variabelen in plaats van formula
import statsmodels.api as sm

# Maak design matrix
X = df_did[['post_treatment', 'treatment_group', 'post_x_treatment']]
X = sm.add_constant(X)  # voeg intercept toe
y = df_did[OUTCOME_VARIABLE]

# Fit model
model = sm.OLS(y, X).fit()

print("DiD Regressie Resultaten:")
print(model.summary())

# Extract DiD coefficient (β3)
did_coefficient = model.params['post_x_treatment']
did_pvalue = model.pvalues['post_x_treatment']
did_ci = model.conf_int().loc['post_x_treatment']

print(f"\n=== DiD RESULTATEN ===")
print(f"DiD Effect: {did_coefficient:.2f}")
print(f"P-waarde: {did_pvalue:.4f}")
print(f"95% Betrouwbaarheidsinterval: [{did_ci[0]:.2f}, {did_ci[1]:.2f}]")
print(f"Significantie: {'***' if did_pvalue < 0.001 else '**' if did_pvalue < 0.01 else '*' if did_pvalue < 0.05 else 'Niet significant'}")

DiD Regressie Resultaten:
                             OLS Regression Results                            
Dep. Variable:     Aantal oprichtingen   R-squared:                       0.167
Model:                             OLS   Adj. R-squared:                  0.162
Method:                  Least Squares   F-statistic:                     34.81
Date:                 Fri, 22 Aug 2025   Prob (F-statistic):           1.61e-20
Time:                         11:39:25   Log-Likelihood:                -4775.0
No. Observations:                  525   AIC:                             9558.
Df Residuals:                      521   BIC:                             9575.
Df Model:                            3                                         
Covariance Type:             nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
cons

## 6. Visualisatie van Resultaten

In [82]:
# 1. Trend lijnen voor alle groepen - GEÏNDEXEERD (Basis 100 = 2015)
group_trends = df_did.groupby(['Jaar', 'group_label'])[OUTCOME_VARIABLE].mean().reset_index()

print("📊 GEÏNDEXEERDE TREND ANALYSE (Basis: 2015 = 100)")
print("=" * 50)

# Bereken index voor elke groep (2015 = 100)
group_trends_indexed = group_trends.copy()
base_year = min(ALL_YEARS)  # 2015

# Voor elke groep, bereken de index
for group in group_trends['group_label'].unique():
    group_data = group_trends[group_trends['group_label'] == group].copy()
    
    # Vind baseline waarde (2015)
    baseline = group_data[group_data['Jaar'] == base_year][OUTCOME_VARIABLE].iloc[0]
    
    # Bereken index (basis 100)
    group_mask = group_trends_indexed['group_label'] == group
    group_trends_indexed.loc[group_mask, 'Index'] = (group_trends_indexed.loc[group_mask, OUTCOME_VARIABLE] / baseline) * 100
    
    print(f"{group}:")
    print(f"  Baseline 2015: {baseline:.0f} oprichtingen")
    print(f"  Index 2022: {group_trends_indexed.loc[group_mask & (group_trends_indexed['Jaar'] == max(ALL_YEARS)), 'Index'].iloc[0]:.1f}")
    print()

# Maak de geïndexeerde grafiek
fig1_indexed = px.line(group_trends_indexed, 
                      x='Jaar', 
                      y='Index', 
                      color='group_label',
                      title=f'DiD Analyse: Geïndexeerde Trends (Basis: {base_year} = 100)',
                      markers=True)

# Voeg horizontale lijn toe voor baseline
fig1_indexed.add_hline(y=100, line_dash="dot", line_color="gray", 
                      annotation_text="Baseline (2015 = 100)")

# Voeg verticale lijn toe voor treatment
fig1_indexed.add_vline(x=TREATMENT_YEAR, line_dash="dash", line_color="red", 
                      annotation_text="Treatment Start")

# Update layout
fig1_indexed.update_layout(
    height=600, 
    width=1000,
    yaxis_title="Index (2015 = 100)",
    xaxis_title="Jaar",
    legend_title="DiD Groepen"
)

# Voeg annotaties toe voor beter begrip


fig1_indexed.show()

# Bereken de relatieve veranderingen
print(f"📈 RELATIEVE VERANDERINGEN (2015 → 2022):")
print("=" * 45)
for group in group_trends_indexed['group_label'].unique():
    group_data = group_trends_indexed[group_trends_indexed['group_label'] == group]
    start_index = group_data[group_data['Jaar'] == base_year]['Index'].iloc[0]
    end_index = group_data[group_data['Jaar'] == max(ALL_YEARS)]['Index'].iloc[0]
    growth = end_index - start_index
    
    trend_symbol = "📈" if growth > 0 else "📉" if growth < 0 else "➡️"
    print(f"{trend_symbol} {group}:")
    print(f"    {start_index:.0f} → {end_index:.1f} ({growth:+.1f} punten)")
    print()

📊 GEÏNDEXEERDE TREND ANALYSE (Basis: 2015 = 100)
Bru/Wal-Alle sectoren:
  Baseline 2015: 653 oprichtingen
  Index 2022: 136.5

Bru/Wal-Bouw:
  Baseline 2015: 2344 oprichtingen
  Index 2022: 92.9

Vlaanderen-Alle sectoren:
  Baseline 2015: 1919 oprichtingen
  Index 2022: 175.8

Vlaanderen-Bouw:
  Baseline 2015: 5465 oprichtingen
  Index 2022: 213.5



📈 RELATIEVE VERANDERINGEN (2015 → 2022):
📈 Bru/Wal-Alle sectoren:
    100 → 136.5 (+36.5 punten)

📉 Bru/Wal-Bouw:
    100 → 92.9 (-7.1 punten)

📈 Vlaanderen-Alle sectoren:
    100 → 175.8 (+75.8 punten)

📈 Vlaanderen-Bouw:
    100 → 213.5 (+113.5 punten)



In [83]:
# 2. Voor/Na vergelijking per groep - GEÏNDEXEERD
print("📊 VOOR/NA VERGELIJKING - GEÏNDEXEERD (Basis: Voor = 100)")
print("=" * 55)

# Bereken gemiddeldes voor voor/na periodes
pre_post_comparison = df_did.groupby(['group_label', 'post_treatment'])[OUTCOME_VARIABLE].mean().reset_index()

# Bereken index voor elke groep (voor period = 100)
pre_post_indexed = pre_post_comparison.copy()
pre_post_indexed['Index'] = 0

for group in pre_post_comparison['group_label'].unique():
    group_data = pre_post_comparison[pre_post_comparison['group_label'] == group]
    
    # Vind baseline (voor treatment)
    baseline = group_data[group_data['post_treatment'] == 0][OUTCOME_VARIABLE].iloc[0]
    
    # Bereken index voor beide periodes
    group_mask = pre_post_indexed['group_label'] == group
    pre_post_indexed.loc[group_mask, 'Index'] = (pre_post_indexed.loc[group_mask, OUTCOME_VARIABLE] / baseline) * 100
    
    # Print resultaten
    voor_index = pre_post_indexed.loc[group_mask & (pre_post_indexed['post_treatment'] == 0), 'Index'].iloc[0]
    na_index = pre_post_indexed.loc[group_mask & (pre_post_indexed['post_treatment'] == 1), 'Index'].iloc[0]
    verschil = na_index - voor_index
    
    trend_symbol = "📈" if verschil > 0 else "📉" if verschil < 0 else "➡️"
    print(f"{trend_symbol} {group}:")
    print(f"    Voor: {voor_index:.0f} | Na: {na_index:.1f} | Verschil: {verschil:+.1f} punten")

# Voeg periode labels toe
pre_post_indexed['Periode'] = pre_post_indexed['post_treatment'].map({
    0: f'Voor Treatment ({min(PRE_TREATMENT_YEARS)}-{max(PRE_TREATMENT_YEARS)})', 
    1: f'Na Treatment ({min(POST_TREATMENT_YEARS)}-{max(POST_TREATMENT_YEARS)})'
})

# Maak de geïndexeerde bar chart
fig2_indexed = px.bar(pre_post_indexed, 
                     x='group_label', 
                     y='Index', 
                     color='Periode',
                     title='Voor vs Na Treatment - Geïndexeerd (Voor = 100)',
                     barmode='group',
                     color_discrete_map={
                         f'Voor Treatment ({min(PRE_TREATMENT_YEARS)}-{max(PRE_TREATMENT_YEARS)})': 'lightblue', 
                         f'Na Treatment ({min(POST_TREATMENT_YEARS)}-{max(POST_TREATMENT_YEARS)})': 'darkblue'
                     })

# Voeg horizontale lijn toe voor baseline
fig2_indexed.add_hline(y=100, line_dash="dot", line_color="gray", 
                      annotation_text="Baseline (Voor = 100)")

fig2_indexed.update_layout(
    height=500, 
    width=1000, 
    xaxis_tickangle=-45,
    yaxis_title="Index (Voor Treatment = 100)",
    xaxis_title="DiD Groepen"
)

# Highlight de treatment groep
fig2_indexed.add_annotation(
    x="Vlaanderen-Bouw", y=250,
    text="🎯 TREATMENT GROEP",
    showarrow=True,
    arrowhead=2,
    arrowcolor="red",
    arrowwidth=2,
    font=dict(size=12, color="red")
)

fig2_indexed.show()

print(f"\n🎯 BELANGRIJKSTE INZICHT:")
print("De treatment groep toont de grootste relatieve toename!")
print("Dit is precies wat we verwachten van een succesvol treatment effect.")

📊 VOOR/NA VERGELIJKING - GEÏNDEXEERD (Basis: Voor = 100)
📈 Bru/Wal-Alle sectoren:
    Voor: 100 | Na: 108.2 | Verschil: +8.2 punten
📉 Bru/Wal-Bouw:
    Voor: 100 | Na: 94.5 | Verschil: -5.5 punten
📈 Vlaanderen-Alle sectoren:
    Voor: 100 | Na: 129.0 | Verschil: +29.0 punten
📈 Vlaanderen-Bouw:
    Voor: 100 | Na: 185.1 | Verschil: +85.1 punten



🎯 BELANGRIJKSTE INZICHT:
De treatment groep toont de grootste relatieve toename!
Dit is precies wat we verwachten van een succesvol treatment effect.


### 💡 **Voordelen van Geïndexeerde Visualisatie (Basis 100)**

#### 🎯 **Waarom indexeren beter is dan absolute waarden:**

1. **Vergelijkbare schaal**: Alle groepen starten op 100, waardoor relatieve veranderingen direct vergelijkbaar zijn
2. **Focus op verandering**: De DiD methode draait om relatieve veranderingen, niet absolute niveaus
3. **Duidelijke interpretatie**: 
   - Index > 100 = groei ten opzichte van basis
   - Index < 100 = krimp ten opzichte van basis
   - Verschil tussen groepen = relatief effect

#### 📊 **Wat de geïndexeerde resultaten tonen:**

- **Vlaanderen-Bouw**: +113.5 punten (2015→2022) en +85.1 punten (voor→na)
- **Sterkste controle (Vlaanderen-Alle sectoren)**: +75.8 punten (2015→2022) en +29.0 punten (voor→na)
- **Zwakste controle (Bru/Wal-Bouw)**: -7.1 punten (2015→2022) en -5.5 punten (voor→na)

#### 🔍 **DiD Effect wordt nu veel duidelijker:**
De treatment groep stijgt **85.1 punten** vs. gemiddeld **~10.6 punten** voor controle groepen = **~75 punten verschil**!

Dit bevestigt het statistische DiD effect van +4,993 oprichtingen, maar toont het nu in relatieve termen.

In [ ]:
# 3. Heatmap van de drie verschil-dimensies
# Creëer een matrix voor de heatmap
heatmap_data = df_did.groupby(['Gewest', 'Sectie', 'post_treatment'])[OUTCOME_VARIABLE].mean().unstack()
heatmap_data.columns = ['Voor Treatment', 'Na Treatment']
heatmap_data['Verschil'] = heatmap_data['Na Treatment'] - heatmap_data['Voor Treatment']

# Creëer subplot voor verschillende perspectieven
fig3 = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Voor Treatment', 'Na Treatment', 'Verschil (Na - Voor)'),
    specs=[[{"type": "heatmap"}, {"type": "heatmap"}, {"type": "heatmap"}]]
)

# Reset index voor plotting
heatmap_plot = heatmap_data.reset_index()
pivot_voor = heatmap_plot.pivot(index='Gewest', columns='Sectie', values='Voor Treatment')
pivot_na = heatmap_plot.pivot(index='Gewest', columns='Sectie', values='Na Treatment')
pivot_verschil = heatmap_plot.pivot(index='Gewest', columns='Sectie', values='Verschil')

# Voeg heatmaps toe
fig3.add_trace(
    go.Heatmap(z=pivot_voor.values, 
               x=[s.split(' ')[0] for s in pivot_voor.columns], 
               y=pivot_voor.index,
               colorscale='Blues',
               showscale=False),
    row=1, col=1
)

fig3.add_trace(
    go.Heatmap(z=pivot_na.values, 
               x=[s.split(' ')[0] for s in pivot_na.columns], 
               y=pivot_na.index,
               colorscale='Blues',
               showscale=False),
    row=1, col=2
)

fig3.add_trace(
    go.Heatmap(z=pivot_verschil.values, 
               x=[s.split(' ')[0] for s in pivot_verschil.columns], 
               y=pivot_verschil.index,
               colorscale='RdBu',
               showscale=True),
    row=1, col=3
)

fig3.update_layout(height=400, width=1200, title_text="Drie Verschil-Dimensies Heatmap")
fig3.show()

### 📖 **Hoe lees je de "Drie Verschil-Dimensies Heatmap"?**

De heatmap visualiseert de kern van de DiD methodologie door drie dimensies te tonen:

#### 🟦 **Panel 1: "Voor Treatment" (2015-2018)**
- **Wat zie je**: Gemiddeld aantal oprichtingen per regio-sector combinatie vóór 2019
- **Kleuren**: Hoe donkerder blauw, hoe meer oprichtingen
- **Interpretatie**: Dit toont de baseline niveaus voor alle groepen

#### 🟦 **Panel 2: "Na Treatment" (2019-2022)** 
- **Wat zie je**: Gemiddeld aantal oprichtingen per regio-sector combinatie vanaf 2019
- **Kleuren**: Hoe donkerder blauw, hoe meer oprichtingen  
- **Interpretatie**: Dit toont de niveaus na de treatment

#### 🔴🔵 **Panel 3: "Verschil (Na - Voor)" - DE BELANGRIJKSTE!**
- **Wat zie je**: Het verschil tussen na en voor treatment voor elke combinatie
- **Kleuren**: 
  - 🔴 **Rood**: Toename in oprichtingen (positief verschil)
  - 🔵 **Blauw**: Afname in oprichtingen (negatief verschil)
  - ⚪ **Wit**: Geen verandering
- **DiD interpretatie**:
  - **Vlaanderen-Bouw** (treatment): Verwacht sterke rode kleur
  - **Andere combinaties** (controle): Verwacht minder intense kleuren

#### 🎯 **Wat zoek je?**
1. **Treatment effect**: Is Vlaanderen-Bouw het roodst (grootste toename)?
2. **Parallel trends**: Hebben andere sectoren in Vlaanderen vergelijkbare kleuren als bouw in andere regio's?
3. **Unieke impact**: Steekt Vlaanderen-Bouw er duidelijk uit?

In [85]:
# PRAKTISCHE LEESWIJZER voor de Heatmap
print("🔍 HEATMAP INTERPRETATIE GIDS")
print("=" * 40)

# Bereken de waarden voor interpretatie
heatmap_values = df_did.groupby(['Gewest', 'Sectie', 'post_treatment'])[OUTCOME_VARIABLE].mean().unstack()
heatmap_values.columns = ['Voor', 'Na'] 
heatmap_values['Verschil'] = heatmap_values['Na'] - heatmap_values['Voor']

print("\n📊 VERSCHIL WAARDEN (Na - Voor):")
print("=" * 40)

# Focus op de belangrijkste combinaties
for region in ['Vlaams Gewest', 'Brussels Hoofdstedelijk Gewest', 'Waals Gewest']:
    print(f"\n🌍 {region}:")
    region_data = heatmap_values.loc[region]
    
    # Zoek bouwsector
    bouw_data = region_data[region_data.index.str.contains('F Bouwnijverheid', na=False)]
    if not bouw_data.empty:
        verschil = bouw_data['Verschil'].iloc[0]
        print(f"   🏗️  Bouwsector: {verschil:+.0f} oprichtingen ({'🔴 TOENAME' if verschil > 0 else '🔵 AFNAME' if verschil < 0 else '⚪ GEEN VERANDERING'})")
    
    # Gemiddelde van andere sectoren
    andere_sectoren = region_data[~region_data.index.str.contains('F Bouwnijverheid', na=False)]
    if not andere_sectoren.empty:
        gem_verschil = andere_sectoren['Verschil'].mean()
        print(f"   🏢 Andere sectoren (gem.): {gem_verschil:+.0f} oprichtingen")

print(f"\n🎯 CONCLUSIE:")
treatment_verschil = heatmap_values.loc['Vlaams Gewest'].loc[heatmap_values.loc['Vlaams Gewest'].index.str.contains('F Bouwnijverheid', na=False), 'Verschil'].iloc[0]
controle_gem = heatmap_values[~heatmap_values.index.get_level_values(1).str.contains('F Bouwnijverheid', na=False)]['Verschil'].mean()

print(f"Treatment (Vlaanderen-Bouw): {treatment_verschil:+.0f}")
print(f"Controle groepen (gemiddeld): {controle_gem:+.0f}")
print(f"DiD Effect: {treatment_verschil - controle_gem:+.0f} (verschil van verschillen)")

if treatment_verschil > controle_gem:
    print("✅ De treatment groep toont een duidelijk sterkere toename!")
else:
    print("❌ De treatment groep toont geen duidelijk sterkere toename.")

🔍 HEATMAP INTERPRETATIE GIDS

📊 VERSCHIL WAARDEN (Na - Voor):

🌍 Vlaams Gewest:
   🏗️  Bouwsector: +5230 oprichtingen (🔴 TOENAME)
   🏢 Andere sectoren (gem.): +647 oprichtingen

🌍 Brussels Hoofdstedelijk Gewest:
   🏗️  Bouwsector: -493 oprichtingen (🔵 AFNAME)
   🏢 Andere sectoren (gem.): +17 oprichtingen

🌍 Waals Gewest:
   🏗️  Bouwsector: +230 oprichtingen (🔴 TOENAME)
   🏢 Andere sectoren (gem.): +129 oprichtingen

🎯 CONCLUSIE:
Treatment (Vlaanderen-Bouw): +5230
Controle groepen (gemiddeld): +264
DiD Effect: +4966 (verschil van verschillen)
✅ De treatment groep toont een duidelijk sterkere toename!


In [86]:
# 🔍 ANALYSE: Waarom is de Treatment groep zo hoog?
print("🤔 MYSTERIE: Waarom heeft Treatment (Vlaanderen-Bouw) zulke hoge waarden?")
print("=" * 70)

# Analyseer de data in detail
print("\n1️⃣ AANTAL OBSERVATIES PER GROEP:")
print("-" * 40)
group_counts = df_did.groupby('group_label').size()
print(group_counts)

print("\n2️⃣ TREATMENT GROEP vs ANDERE SECTOREN IN VLAANDEREN:")
print("-" * 50)

# Vergelijk bouw vs andere sectoren in Vlaanderen
vlaanderen_data = df_did[df_did['Gewest'] == 'Vlaams Gewest']

print("🏗️ BOUWSECTOR in Vlaanderen:")
bouw_vl = vlaanderen_data[vlaanderen_data['Sectie'] == 'F Bouwnijverheid']
if not bouw_vl.empty:
    print(f"   Voor: {bouw_vl[bouw_vl['post_treatment']==0][OUTCOME_VARIABLE].mean():.0f} oprichtingen")
    print(f"   Na:   {bouw_vl[bouw_vl['post_treatment']==1][OUTCOME_VARIABLE].mean():.0f} oprichtingen")

print(f"\n🏢 ANDERE SECTOREN in Vlaanderen (top 5):")
andere_vl = vlaanderen_data[vlaanderen_data['Sectie'] != 'F Bouwnijverheid']
sector_gemiddelden = andere_vl.groupby('Sectie')[OUTCOME_VARIABLE].mean().sort_values(ascending=False)
for i, (sector, gem) in enumerate(sector_gemiddelden.head().items()):
    print(f"   {i+1}. {sector[:30]}{'...' if len(sector) > 30 else ''}: {gem:.0f}")

print(f"\n3️⃣ WAAROM ZO HOOG? MOGELIJKE VERKLARINGEN:")
print("-" * 45)

# Bereken totaal voor alle andere sectoren samen
total_andere = andere_vl[OUTCOME_VARIABLE].sum() / len(andere_vl['Jaar'].unique())  # gemiddeld per jaar
bouw_totaal = bouw_vl[OUTCOME_VARIABLE].sum() / len(bouw_vl['Jaar'].unique())

print(f"✅ Bouwsector alleen: {bouw_totaal:.0f} oprichtingen/jaar")
print(f"📊 Alle andere sectoren samen: {total_andere:.0f} oprichtingen/jaar")

if bouw_totaal > total_andere:
    print(f"\n🎯 VERKLARING: De bouwsector is inderdaad een ZEER GROTE sector!")
    print(f"   • Bouw is waarschijnlijk een van de grootste sectoren qua nieuwe bedrijven")
    print(f"   • Veel kleine aannemers, ZZP'ers, en gespecialiseerde bedrijven")
    print(f"   • Lage instapdrempel voor nieuwe bedrijven")
else:
    print(f"\n🎯 VERKLARING: Dit komt door de AGGREGATIE in de grafiek!")
    print(f"   • Treatment toont 1 sector (bouw)")  
    print(f"   • Controle toont GEMIDDELDE van {len(CONTROL_SECTORS)} sectoren")

print(f"\n4️⃣ VISUALISATIE EFFECT:")
print("-" * 25)
print("In de grafiek zie je gemiddeldes per groep:")
print("• Treatment: Gemiddelde van 1 sector (bouw)")
print("• Control Sector: Gemiddelde van 21 sectoren") 
print("• Dit maakt vergelijking mogelijk, maar kan verwarrend lijken!")

🤔 MYSTERIE: Waarom heeft Treatment (Vlaanderen-Bouw) zulke hoge waarden?

1️⃣ AANTAL OBSERVATIES PER GROEP:
----------------------------------------
group_label
Bru/Wal-Alle sectoren       333
Bru/Wal-Bouw                 16
Vlaanderen-Alle sectoren    168
Vlaanderen-Bouw               8
dtype: int64

2️⃣ TREATMENT GROEP vs ANDERE SECTOREN IN VLAANDEREN:
--------------------------------------------------
🏗️ BOUWSECTOR in Vlaanderen:
   Voor: 6143 oprichtingen
   Na:   11373 oprichtingen

🏢 ANDERE SECTOREN in Vlaanderen (top 5):
   1. M Vrije beroepen en wetenschap...: 11895
   2. G Groot- en detailhandel/ repa...: 8746
   3. N Administratieve en ondersteu...: 4724
   4. S Overige diensten: 4476
   5. Q Menselijke gezondheidszorg e...: 3772

3️⃣ WAAROM ZO HOOG? MOGELIJKE VERKLARINGEN:
---------------------------------------------
✅ Bouwsector alleen: 8758 oprichtingen/jaar
📊 Alle andere sectoren samen: 53658 oprichtingen/jaar

🎯 VERKLARING: Dit komt door de AGGREGATIE in de grafiek!
   

In [87]:
# 📊 ALTERNATIEVE VISUALISATIE: Sectoren in perspectief
print("📊 SECTOREN IN PERSPECTIEF - Vlaanderen")
print("=" * 45)

# Maak een vergelijking van alle sectoren in Vlaanderen
vlaanderen_sectoren = df_did[df_did['Gewest'] == 'Vlaams Gewest'].groupby('Sectie')[OUTCOME_VARIABLE].mean().sort_values(ascending=False)

# Toon top 10 sectoren
print("🏆 TOP 10 SECTOREN in Vlaanderen (gemiddeld aantal oprichtingen):")
print("-" * 60)
for i, (sector, oprichtingen) in enumerate(vlaanderen_sectoren.head(10).items(), 1):
    is_bouw = "🏗️" if "F Bouwnijverheid" in sector else "🏢"
    sector_kort = sector.split(' ')[0] + " " + sector.split(' ')[1] if len(sector.split(' ')) > 1 else sector
    print(f"{i:2d}. {is_bouw} {sector_kort:<25}: {oprichtingen:>7.0f} oprichtingen")

print(f"\n💡 BELANGRIJKE INZICHTEN:")
print("-" * 25)
bouw_rang = list(vlaanderen_sectoren.index).index('F Bouwnijverheid') + 1
print(f"• Bouwsector staat op plaats {bouw_rang} van {len(vlaanderen_sectoren)} sectoren")
print(f"• Bouw is inderdaad een grote sector, maar niet de grootste")
print(f"• De grafiek toont GEMIDDELDES per groep, niet totalen")

# Maak een nieuwe grafiek die dit beter toont
import plotly.express as px

# Data voor betere visualisatie
vl_voor_na = df_did[df_did['Gewest'] == 'Vlaams Gewest'].groupby(['Sectie', 'post_treatment'])[OUTCOME_VARIABLE].mean().reset_index()
vl_voor_na['Periode'] = vl_voor_na['post_treatment'].map({0: 'Voor (2015-2018)', 1: 'Na (2019-2022)'})
vl_voor_na['Sector_kort'] = vl_voor_na['Sectie'].str.split(' ').str[0]
vl_voor_na['Is_Bouw'] = vl_voor_na['Sectie'].str.contains('F Bouwnijverheid')

# Sorteer op gemiddelde waarde
sector_order = vl_voor_na.groupby('Sector_kort')[OUTCOME_VARIABLE].mean().sort_values(ascending=True).index

fig_sectors = px.bar(vl_voor_na, 
                    x=OUTCOME_VARIABLE, 
                    y='Sector_kort',
                    color='Periode',
                    title='Alle Sectoren in Vlaanderen: Voor vs Na Treatment',
                    orientation='h',
                    category_orders={'Sector_kort': sector_order},
                    color_discrete_map={'Voor (2015-2018)': 'lightblue', 'Na (2019-2022)': 'darkblue'})

# Highlight bouwsector
fig_sectors.update_layout(height=800, width=1000)
fig_sectors.add_annotation(
    x=8000, y='F',
    text="🏗️ TREATMENT SECTOR",
    showarrow=True,
    arrowhead=2,
    arrowcolor="red",
    arrowwidth=2,
    font=dict(size=14, color="red")
)

fig_sectors.show()

📊 SECTOREN IN PERSPECTIEF - Vlaanderen
🏆 TOP 10 SECTOREN in Vlaanderen (gemiddeld aantal oprichtingen):
------------------------------------------------------------
 1. 🏢 M Vrije                  :   11895 oprichtingen
 2. 🏗️ F Bouwnijverheid         :    8758 oprichtingen
 3. 🏢 G Groot-                 :    8746 oprichtingen
 4. 🏢 N Administratieve        :    4724 oprichtingen
 5. 🏢 S Overige                :    4476 oprichtingen
 6. 🏢 Q Menselijke             :    3772 oprichtingen
 7. 🏢 J Informatie             :    3765 oprichtingen
 8. 🏢 I Verschaffen            :    3630 oprichtingen
 9. 🏢 C Industrie              :    2843 oprichtingen
10. 🏢 R Kunst,                 :    2514 oprichtingen

💡 BELANGRIJKE INZICHTEN:
-------------------------
• Bouwsector staat op plaats 2 van 22 sectoren
• Bouw is inderdaad een grote sector, maar niet de grootste
• De grafiek toont GEMIDDELDES per groep, niet totalen


## 7. Statistische Testing en Robuustheid

In [88]:
# Parallel trends assumptie test
# Test of de trends in de pre-treatment periode parallel zijn

pre_treatment_data = df_did[df_did['post_treatment'] == 0]

# Test voor parallel trends met lineaire tijd trend
# Gebruik OLS in plaats van formula om kolom naam problemen te vermijden
import statsmodels.api as sm

# Maak design matrix voor parallel trends test
X_parallel = pre_treatment_data[['Jaar', 'treatment_group']].copy()
X_parallel['Jaar_x_treatment'] = X_parallel['Jaar'] * X_parallel['treatment_group']
X_parallel = sm.add_constant(X_parallel)
y_parallel = pre_treatment_data[OUTCOME_VARIABLE]

# Fit parallel trends model
parallel_model = sm.OLS(y_parallel, X_parallel).fit()

print("Parallel Trends Test (Pre-treatment periode):")
print(parallel_model.summary())

# Test statistiek voor parallel trends
interaction_coef = parallel_model.params.get('Jaar_x_treatment', 0)
interaction_pvalue = parallel_model.pvalues.get('Jaar_x_treatment', 1)

print(f"\nParallel Trends Test:")
print(f"Interactie coëfficiënt (Jaar x Treatment): {interaction_coef:.4f}")
print(f"P-waarde: {interaction_pvalue:.4f}")
print(f"Parallel trends assumptie: {'VOLDAAN' if interaction_pvalue > 0.05 else 'GESCHONDEN'}")
print(f"(p > 0.05 betekent dat trends parallel zijn)")

Parallel Trends Test (Pre-treatment periode):
                             OLS Regression Results                            
Dep. Variable:     Aantal oprichtingen   R-squared:                       0.097
Model:                             OLS   Adj. R-squared:                  0.087
Method:                  Least Squares   F-statistic:                     9.246
Date:                 Fri, 22 Aug 2025   Prob (F-statistic):           7.90e-06
Time:                         11:39:25   Log-Likelihood:                -2333.9
No. Observations:                  261   AIC:                             4676.
Df Residuals:                      257   BIC:                             4690.
Df Model:                            3                                         
Covariance Type:             nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

In [89]:
# Robuustheid test: verschillende tijdvensters
print("Robuustheid Analyse - Verschillende Tijdvensters:")
print("=" * 50)

# Test verschillende cut-off jaren
cutoff_years = [2018, 2019, 2020]
robustness_results = []

for cutoff in cutoff_years:
    df_temp = df_did.copy()
    df_temp['post_alt'] = (df_temp['Jaar'] >= cutoff).astype(int)
    df_temp['post_x_treatment_alt'] = df_temp['post_alt'] * df_temp['treatment_group']
    
    # Gebruik OLS in plaats van formula
    X_rob = df_temp[['post_alt', 'treatment_group', 'post_x_treatment_alt']]
    X_rob = sm.add_constant(X_rob)
    y_rob = df_temp[OUTCOME_VARIABLE]
    
    model_alt = sm.OLS(y_rob, X_rob).fit()
    
    coef = model_alt.params['post_x_treatment_alt']
    pval = model_alt.pvalues['post_x_treatment_alt']
    
    robustness_results.append({
        'cutoff_year': cutoff,
        'did_effect': coef,
        'p_value': pval,
        'significant': pval < 0.05
    })
    
    print(f"Cutoff jaar {cutoff}: DiD Effect = {coef:.2f}, p-waarde = {pval:.4f}")

robustness_df = pd.DataFrame(robustness_results)
print(f"\nRobuustheid samenvatting:")
print(robustness_df)

Robuustheid Analyse - Verschillende Tijdvensters:
Cutoff jaar 2018: DiD Effect = 4408.61, p-waarde = 0.0060
Cutoff jaar 2019: DiD Effect = 4992.62, p-waarde = 0.0013
Cutoff jaar 2020: DiD Effect = 4724.06, p-waarde = 0.0032

Robuustheid samenvatting:
   cutoff_year   did_effect   p_value  significant
0         2018  4408.606701  0.006002         True
1         2019  4992.624199  0.001286         True
2         2020  4724.058544  0.003211         True
Cutoff jaar 2020: DiD Effect = 4724.06, p-waarde = 0.0032

Robuustheid samenvatting:
   cutoff_year   did_effect   p_value  significant
0         2018  4408.606701  0.006002         True
1         2019  4992.624199  0.001286         True
2         2020  4724.058544  0.003211         True


In [90]:
# Finale samenvatting van resultaten
print("="*60)
print("FINALE DiD ANALYSE RESULTATEN")
print("="*60)

print(f"\n📊 TREATMENT DEFINITIE:")
print(f"   • Jaar: {TREATMENT_YEAR}")
print(f"   • Regio: {TREATMENT_REGION}")
print(f"   • Sector: {TREATMENT_SECTOR}")
print(f"   • Outcome: {OUTCOME_VARIABLE}")

print(f"\n📈 HOOFD RESULTATEN:")
print(f"   • DiD Effect: {did_coefficient:.2f} oprichtingen")
print(f"   • Percentage effect: {percentage_effect:.2f}%")
print(f"   • P-waarde: {did_pvalue:.4f}")
print(f"   • 95% CI: [{did_ci[0]:.2f}, {did_ci[1]:.2f}]")
print(f"   • Significantie: {'***' if did_pvalue < 0.001 else '**' if did_pvalue < 0.01 else '*' if did_pvalue < 0.05 else 'Niet significant'}")

print(f"\n🔍 ASSUMPTIE CHECKS:")
print(f"   • Parallel trends: {'✅ VOLDAAN' if interaction_pvalue > 0.05 else '❌ GESCHONDEN'} (p={interaction_pvalue:.4f})")

consistent_sign = all(r['did_effect'] > 0 for r in robustness_results) or all(r['did_effect'] < 0 for r in robustness_results)
print(f"   • Robuustheid: {'✅ CONSISTENT' if consistent_sign else '❌ INCONSISTENT'} over verschillende cutoffs")

print(f"\n💡 CONCLUSIE:")
if did_pvalue < 0.05:
    direction = "positief" if did_coefficient > 0 else "negatief"
    print(f"   Er is een statistisch significant {direction} effect van de treatment")
    print(f"   op het aantal oprichtingen in de bouwsector in Vlaanderen.")
else:
    print(f"   Er is geen statistisch significant effect gevonden van de treatment.")

print("\n" + "="*60)

FINALE DiD ANALYSE RESULTATEN

📊 TREATMENT DEFINITIE:
   • Jaar: 2019
   • Regio: Vlaams Gewest
   • Sector: F Bouwnijverheid
   • Outcome: Aantal oprichtingen

📈 HOOFD RESULTATEN:
   • DiD Effect: 4992.62 oprichtingen
   • Percentage effect: 81.27%
   • P-waarde: 0.0013
   • 95% CI: [1962.40, 8022.85]
   • Significantie: **

🔍 ASSUMPTIE CHECKS:
   • Parallel trends: ✅ VOLDAAN (p=0.6218)
   • Robuustheid: ✅ CONSISTENT over verschillende cutoffs

💡 CONCLUSIE:
   Er is een statistisch significant positief effect van de treatment
   op het aantal oprichtingen in de bouwsector in Vlaanderen.



## 📋 Belangrijkste Bevindingen en Verbeteringen

### 🔍 **Methodologische Verbeteringen:**
- **Uitgebreide controle groep**: Gebruikt nu alle 21 sectoren (behalve bouw) als controle in plaats van slechts 3 sectoren
- **Grotere steekproef**: 525 observaties vs. kleinere dataset in eerdere versie
- **Robuustere vergelijking**: Bouwsector vs. hele economie geeft betere baseline

### 📊 **Sterke Empirische Resultaten:**
- **Zeer significant effect**: p-waarde = 0.0013 (α < 0.01)
- **Groot effect**: +4,993 extra oprichtingen (+81% t.o.v. baseline)
- **Robuust**: Effect blijft significant over verschillende tijdvensters (2018-2020)
- **Valide assumptie**: Parallel trends assumptie wordt niet geschonden (p = 0.6218)

### 🎯 **Economische Interpretatie:**
1. **Treatment effect**: De bouwsector in Vlaanderen laat vanaf 2019 een substantiële toename zien in bedrijfsoprichtingen
2. **Relatieve prestatie**: Dit effect is veel groter dan de algemene economische trends in andere sectoren
3. **Regionale specificiteit**: Het effect is specifiek voor Vlaanderen vs. andere gewesten

### ⚠️ **Kanttekeningen:**
- Analyse toont correlatie, niet noodzakelijk causatie
- Mogelijke confounding factors niet gecontroleerd (economische conjunctuur, beleid, etc.)
- Kleine sample size voor treatment groep (4 observaties voor + 4 na)

### 🔮 **Vervolgonderzoek:**
- Onderzoek naar mogelijke verklarende mechanismen
- Analyse van andere outcome variabelen (aantal schrappingen, btw-plichtigen)
- Uitbreiding naar maandcijfers voor meer granulariteit